In [1]:
import os
import win32com.client as win32
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
def StreamProp_input(stream):
    stream_element = stream.Elements("Input")
    FlashtypeChoice = stream_element.Elements("MIXED_SPEC").Elements("MIXED").Value        
    Temp = stream_element.Elements("TEMP").Elements("MIXED").Value
    Pressure = stream_element.Elements("PRES").Elements("MIXED").Value
    VaporFraction = stream_element.Elements("VFRAC").Elements("MIXED").Value 
    TotalFlowRate = stream_element.Elements("TOTFLOW").Value

    CompoundNameList = []
    CompoundFracList = []
    CompoundNameNode = stream_element.Elements("FLOW").Elements("MIXED").Elements
    for CompoundName in CompoundNameNode:
        CompoundNameList.append(CompoundName.Name)
        CompoundFracList.append(CompoundName.Value)
    comp_list = {comp : value for comp,value in zip(CompoundNameList, CompoundFracList)}

    Dictionary = {
        "FlashtypeChoice":FlashtypeChoice,
        "Temp":Temp,
        "Pressure":Pressure,
        "VaporFraction":VaporFraction,
        "TotalFlowRate":TotalFlowRate,
        "CompoundNameNfrac":comp_list,
    }
    print(json.dumps(Dictionary, indent=4))
    return Dictionary

def StreamProp_output(stream):
    stream_element = stream.Elements("Output")
    Compoundlist = stream_element.Elements("MOLEFRAC").Elements("MIXED").Elements
    CompoundNameList = []
    MoleFlowList = []
    MassFlowList = []
    MoleFracList = []
    MassFracList = []
    LiquidConcentrationList = []
    VaporConcentrationList = []
    
    for compound in Compoundlist:
        Compoundname = compound.Name
        CompoundNameList.append(Compoundname)
        MoleFlowList.append(stream_element.Elements("STR_MAIN").Elements("MOLEFLOW").Elements("MIXED").Elements(Compoundname).Value)
        MassFlowList.append(stream_element.Elements("STR_MAIN").Elements("MASSFLOW").Elements("MIXED").Elements(Compoundname).Value)
        MoleFracList.append(stream_element.Elements("MOLEFRAC").Elements("MIXED").Elements(Compoundname).Value)
        MassFracList.append(stream_element.Elements("MASSFRAC").Elements("MIXED").Elements(Compoundname).Value)
        LiquidConcentrationList.append(stream_element.Elements("X").Elements(Compoundname).Value)
        VaporConcentrationList.append(stream_element.Elements("Y").Elements(Compoundname).Value)

    VaporFraction = stream_element.Elements("STR_MAIN").Elements("VFRAC").Value 
    LiquidFraction = stream_element.Elements("STR_MAIN").Elements("LFRAC").Value 
    SolidFraction = stream_element.Elements("STR_MAIN").Elements("SFRAC").Value 

    Dictionary = {       
        "CompoundNameList":CompoundNameList,
        "MoleFlowList":MoleFlowList,
        "MassFlowList":MassFlowList,
        "MoleFracList":MoleFracList,
        "MassFracList":MassFracList,
        "LiquidConcentrationList":LiquidConcentrationList,
        "VaporConcentrationList":VaporConcentrationList,

        "VaporFraction":VaporFraction,
        "LiquidFraction":LiquidFraction,        
        "SolidFraction":SolidFraction,
    }
    print(json.dumps(Dictionary, indent=4))
    return Dictionary

In [3]:
filename = 'MEA_ex.bkp'
# Aspen open
sim = win32.gencache.EnsureDispatch("Apwn.Document")
sim.InitFromArchive2(os.path.abspath(filename))
sim.Visible = True

In [4]:
# block, stream define
MyBlocks = sim.Tree.Elements("Data").Elements("Blocks")
MyStreams = sim.Tree.Elements("Data").Elements("Streams")

In [5]:
# Block Setting
cooler = MyBlocks.Elements('COOLER')
# Block property
cooling_T = cooler.Elements("Input").Elements("TEMP")
cooling_T.Value = 37

# Block Setting
HX1 = MyBlocks.Elements("HX1")
# Block property
HX1_T  = HX1.Elements("Input").Elements("TEMP")
HX1_T.Value = 90

In [7]:
# Stream control
feed = MyStreams.Elements("MEAMU")
feed_prop = StreamProp_input(feed)

#total flow
F_feed = feed.Elements("Input").Elements("TOTFLOW").Elements("MIXED")
F_feed.Value = 2

#Comp. flow
# F_MEA = feed.Elements("Input").Elements("FLOW").Elements("MIXED").Elements("MEA")
# F_MEA.Value = 1

{
    "FlashtypeChoice": "TP",
    "Temp": 20.0,
    "Pressure": 1.05,
    "VaporFraction": null,
    "TotalFlowRate": 1.0,
    "CompoundNameNfrac": {
        "MEA": 1.0,
        "H2O": null,
        "CO2": null,
        "H2S": null,
        "O2": null,
        "N2": null,
        "MEA+": null,
        "MEACOO-": null,
        "HCO3-": null,
        "CO3-2": null,
        "HS-": null,
        "S-2": null,
        "H3O+": null,
        "OH-": null
    }
}


In [8]:
# Simulation run
sim.Run2()

# Simulation reinitialization
# sim.Reinit()

In [9]:
#Block output
HX1 = MyBlocks.Elements("HX1")
HX1_HD = HX1.Elements("Output").Elements("QCALC").Value

# stream output
Af_mixer = MyStreams.Elements("MIXED")
Af_mixer_prop=StreamProp_output(Af_mixer)

{
    "CompoundNameList": [
        "MEA",
        "H2O",
        "CO2",
        "H2S",
        "O2",
        "N2",
        "MEA+",
        "MEACOO-",
        "HCO3-",
        "CO3-2",
        "HS-",
        "S-2",
        "H3O+",
        "OH-"
    ],
    "MoleFlowList": [
        6058.62138,
        59995.0038,
        0.0180040452,
        0.0,
        0.0,
        0.0,
        917.426895,
        880.811764,
        20.3672002,
        7.84323265,
        0.0,
        0.0,
        7.63252653e-07,
        0.56146617
    ],
    "MassFlowList": [
        370083.132,
        1080826.79,
        0.792354429,
        0.0,
        0.0,
        0.0,
        56964.0543,
        91680.2878,
        1242.75951,
        470.674744,
        0.0,
        0.0,
        1.45191033e-05,
        9.54935485
    ],
    "MoleFracList": [
        0.0892540223,
        0.883830672,
        2.65230875e-07,
        0.0,
        0.0,
        0.0,
        0.0135152926,
        0.0129758881,
        0.000300044

In [10]:
# Aspen close
AspenFileName = sim.FullName
sim.Close(AspenFileName)